In [ ]:
from selenium import webdriver
import time
import csv

chromedriver_path = "/chromedriver.exe"
# Create a new instance of the Chrome driver
driver = webdriver.Chrome(executable_path=chromedriver_path)

home_url = "https://www.vlr.gg/stats/?event_group_id=all&event_id=all&region=na&country=all&min_rounds=200&min_rating=1550&agent=all&map_id=all&timespan=all"
driver.get(home_url)
html_page = driver.page_source
soup = BeautifulSoup(html_page,"lxml")
links = [x['href'] for x in soup.find_all('a') if x.has_attr('href') and x['href'].find('/player') > -1 ]

In [ ]:
html_page = driver.page_source
soup = BeautifulSoup(html_page,"lxml")
parent_node = soup.find('div', {"class":"player-summary-container-2"})

In [ ]:
wfcard_nodes = [x for x in parent_node.find_all('div', {"class": "wf-card"})]

In [ ]:
wfcard_nodes[1].find('span').text

In [ ]:
def grab_total_winnings(html_page):
    soup = BeautifulSoup(html_page,"lxml")
    parent_node = soup.find('div', {"class":"player-summary-container-2"})
    wfcard_nodes = [x for x in parent_node.find_all('div', {"class": "wf-card"})]
    for panel in wfcard_nodes:
        if panel.text.find('Total Winnings') > -1:
            winnings = panel.find('span').text
            return winnings

In [ ]:
soup = BeautifulSoup(html_page,"lxml")
parent_node = soup.find('div', {"class":"player-summary-container-2"})
wfcard_nodes = [x for x in parent_node.find_all('div', {"class": "wf-card"})]
for panel in wfcard_nodes:
    if panel.text.find('Total Winnings') > -1:
        winnings = panel.find('span').text
        print(winnings)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
base_url = 'https://www.vlr.gg'
payload=[]
for player_link in tqdm(links):
    url = f"{base_url}{player_link}"
    driver.get(url)
    html_page = driver.page_source
    winnings = grab_total_winnings(html_page)
    payload.append({'player':player_link,'winnings':winnings})
    time.sleep(5)

In [77]:
import pandas as pd
#df = pd.DataFrame(payload)    
#df['player_short'] = df.player.apply(lambda x: x.split('/')[-1])
#df.to_csv('player_winnings.csv')

df2 = pd.DataFrame(payload2)   
df2['player_short'] = df2.player.apply(lambda x: x.split('/')[-1])
df2.to_csv('player_winnings_version2.csv')
df2.head()

,player,winnings,player_short
0,/player/2450/rustun,"$3,725",rustun
1,/player/3998/bewd,$731,bewd
2,/player/3062/benita,"$35,520",benita
3,/player/52/joshrt,$805,joshrt
4,/player/3111/laisha,$0,laisha


In [88]:
#df.head()
tail_list = set(links) - set(df.player.to_list())
print(len(tail_list))

132


In [89]:
base_url = 'https://www.vlr.gg'
payload2=[]
for player_link in tqdm(tail_list):
    url = f"{base_url}{player_link}"
    driver.get(url)
    html_page = driver.page_source
    winnings = grab_total_winnings(html_page)
    payload2.append({'player':player_link,'winnings':winnings})
    time.sleep(5)
df2 = pd.DataFrame(payload2)   
df2['player_short'] = df2.player.apply(lambda x: x.split('/')[-1])
df2.to_csv('player_winnings_version3.csv')

In [91]:
#df1 = pd.read_csv("player_winnings.csv")
#print(df1.shape)
#df2 = pd.read_csv("player_winnings_version2.csv")
#print(df2.shape)
#df = pd.concat([df1,df2])
#df_full.to_csv("player_winnings_full.csv")
df_full = pd.concat([df,df2])
df_full.shape
df_full.to_csv("player_winnings_full.csv")